In [ ]:
import numpy as np
np.random.seed(42)
import pandas as pd
import os,sys,time,datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

import warnings
#warnings.filterwarnings('ignore')

import os
os.environ['OMP_NUM_THREADS'] = '4'

DataBaseDir = '../../data/version1'
InputDir = '%s/l0/kfold' % DataBaseDir
OutputDir = '%s/l1' % DataBaseDir
kfold = 3
strategy = 'bi-gru'
# load data
start = time.time()
valid_dfs = []
for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir, fold)
    valid = pd.read_csv('%s/valid.csv' % FoldInputDir).reset_index(drop= True)#.sample(frac= 0.1)
    ## for valid/holdout data set
    if(fold == 0):
        HoldoutData = pd.read_csv('%s/holdout.csv' % FoldInputDir).reset_index(drop= True)#.sample(frac= 0.1)
        TestData = pd.read_csv('%s/test.csv' % FoldInputDir).reset_index(drop= True)#.sample(frac= 0.1)
    valid['fold'] = fold
    valid_dfs.append(valid)
    print('load data for fold %s done.' % fold)
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True)
end = time.time()
print('load data done, train %s holdout %s, time elapsed %s' % (len(TrainData), len(HoldoutData), (end - start)))
##
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')

def LoadEmbeddingVectors(f):
    ## debug
    k = 1000
    EmbeddingDict = {}
    with open(f, 'r') as i_file:
        for line in i_file:
            #if(k == 0):
            #    break
            w, coe_vec= get_coefs(*line.rstrip().rsplit(' '))
            EmbeddingDict[w] = coe_vec
            k -= 1
    i_file.close()
    return EmbeddingDict

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

targets = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
EmbeddingFile = '../../data/raw/crawl-300d-2M.vec'
max_features = 30000
maxlen = 100
#max_features = 3000
#maxlen = 10
embed_size = 300
batch_size = 16
epochs = 2
start = time.time()
EmbeddingIndex = LoadEmbeddingVectors(EmbeddingFile)
end = time.time()
print('load embedding features done, corpus size %s, time elapsed %s' % (len(EmbeddingIndex), (end - start)))

def get_model(embedding_matrix):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(80, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

##
cv_score = .0
start = time.time()
pred_cols = ['%s_%s' % (strategy, c) for c in targets]
for c in pred_cols:
    HoldoutData[c] = .0
    TestData[c] = .0
for fold in range(kfold):
    print('====== fold %s ======\n' % fold)
    FoldData = {
        'train': TrainData[TrainData['fold'] != fold],
        'valid': TrainData[TrainData['fold'] == fold],
        'holdout': HoldoutData,
        'test': TestData
    }
    for c in pred_cols:
        FoldData['valid'][c] = .0
        FoldData['holdout'][c] = .0
        FoldData['test'][c] = .0
    ## tokenize with entire corpus composed by train/valid/holdout
    tokenizer = text.Tokenizer(num_words= max_features)
    EntireCorpus = list(FoldData['train']['comment_text'].values) + list(FoldData['valid']['comment_text'].values) + list(FoldData['holdout']['comment_text'].values) + list(FoldData['test']['comment_text'].values)
    tokenizer.fit_on_texts(EntireCorpus)
    X_train = tokenizer.texts_to_sequences(FoldData['train']['comment_text'].values)
    X_valid = tokenizer.texts_to_sequences(FoldData['valid']['comment_text'].values)
    X_holdout = tokenizer.texts_to_sequences(FoldData['holdout']['comment_text'].values)
    X_test = tokenizer.texts_to_sequences(FoldData['test']['comment_text'].values)
    X_train = sequence.pad_sequences(X_train, maxlen= maxlen)
    X_valid = sequence.pad_sequences(X_valid, maxlen= maxlen)
    X_holdout = sequence.pad_sequences(X_holdout, maxlen= maxlen)
    X_test = sequence.pad_sequences(X_test, maxlen= maxlen)
    Y_train = FoldData['train'][targets].values
    Y_valid = FoldData['valid'][targets].values
    #Y_holdout = FoldData['holdout'][targets].values
    end = time.time()
    print('token done, time elapsed %s.' % (end - start))
    ## embedding with pre-trained embedding library
    word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.zeros((nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = EmbeddingIndex.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    end = time.time()
    print('embedding done, time elapsed %s.' % (end - start))
    ## construct bi-gru model
    start = time.time()
    model = get_model(embedding_matrix)
    RocAuc = RocAucEvaluation(validation_data= (X_valid, Y_valid), interval=1)
    hist = model.fit(X_train, Y_train, 
                     batch_size= batch_size, 
                     epochs= epochs, 
                     validation_data= (X_valid, Y_valid),
                     callbacks=[RocAuc], verbose=2)
    end = time.time()
    print('fitting done, time elapsed %s.' % (end - start))
    ## predict for valid
    pred_valid = model.predict(X_valid, batch_size=1024)
    FoldData['valid'][pred_cols] = pred_valid
    ## predict for holdout
    pred_holdout = model.predict(X_holdout, batch_size=1024)
    FoldData['holdout'][pred_cols] = pred_holdout
    HoldoutData[pred_cols] += pred_holdout
    ## predict for test
    pred_test = model.predict(X_test, batch_size=1024)
    FoldData['test'][pred_cols] = pred_test
    TestData[pred_cols] += pred_test
    end = time.time()
    print('predict done, time elapsed %s.' % (end - start))
    ## evaluate
    print(FoldData['valid'][pred_cols].isnull().sum(axis= 0))
    score = roc_auc_score(FoldData['valid'][targets], FoldData['valid'][pred_cols])
    cv_score += score
    print('fold %s, score %.5f' % (fold, score))
    ## output
    FoldOutputDir = '%s/kfold/%s' % (OutputDir, fold)
    if(os.path.exists(FoldOutputDir) == False):
        os.makedirs(FoldOutputDir)
    for mod in ['valid', 'holdout', 'test']:
        if(mod == 'test'):
            out_cols = ['id']
            out_cols.extend(pred_cols)
        else:
            out_cols = pred_cols.copy()
            out_cols.extend(targets)
        FoldData[mod][out_cols].to_csv('%s/%s.csv' % (FoldOutputDir, mod),float_format='%.6f', index= False) 
    end = time.time()
    print('output done, time elapsed %s.\n' % (end - start))
cv_score /= kfold
HoldoutData[pred_cols] /= kfold
TestData[pred_cols] /= kfold
holdout_score = roc_auc_score(HoldoutData[targets], HoldoutData[pred_cols])
end = time.time()
print('\n================')
print('cv score %.5f, holdout score %.5f, time elapsed %s' % (cv_score, holdout_score, (end - start)))
print('================')

## submit
sub = TestData[['id']].copy()
sub[targets] = TestData[pred_cols]
OutputFileName = '%s_submit_%s' % (strategy, datetime.datetime.now().strftime("%Y-%m-%d"))
SubmitDir = '%s/l0/submit' % DataBaseDir
if(os.path.exists(SubmitDir) == False):
    os.makedirs(SubmitDir)
sub.to_csv('%s/%s.csv' % (SubmitDir, OutputFileName), float_format='%.6f', index=False)
print('zip %s/%s.zip %s/%s.csv' % (SubmitDir, OutputFileName, SubmitDir, OutputFileName))
os.system('zip %s/%s.zip %s/%s.csv' % (SubmitDir, OutputFileName, SubmitDir, OutputFileName))

print('\n======================')
print("CV score %.4f, Holdout score %.4f" % (cv_score, holdout_score))
print('======================\n')

load data for fold 0 done.
load data for fold 1 done.
load data for fold 2 done.
load data done, train 143591 holdout 15980, time elapsed 0.9921886920928955
